## How to use Pandas more efficiently

The Pandas library is the tool that almost every data scientist will use every day. It is an awesome tool to explore and work with data.

In this notebook, we would like to share a few good tricks such as using built-in functions and adopting vectorized operations that can speed up our pandas code and improve our productivity. The agenda would be:

1. Selecting Rows and Columns using **.iloc[]** function is faster

2. To iterate through all of rows of a dataframe, in terms of efficiency, vectorizing over pandas series > **.apply()** > **.iterrows()**

3. Try to use more built-in functions such as **groupby()** functions. It could also speed up the process than our coding from scratch.

In the following notebook, we will use the following dataset from Kaggle:


[Crime in Chicago](https://www.kaggle.com/datasets/onlyrohit/crimes-in-chicago?resource=download)

We took a random sample from it and the csv file is named **crimes_in_Chicago_subset.csv** which contains 36082 data points.

In [1]:
import pandas as pd
import numpy as np
df_data = pd.read_csv('../data/crimes_in_Chicago_subset.csv')

In [2]:
df_data.shape

(36082, 22)

### 1. Select rows and columns efficiently

In [3]:
%timeit -r5 -n10 df_data.loc[range(0,600)]

The slowest run took 5.21 times longer than the fastest. This could mean that an intermediate result is being cached.
971 µs ± 831 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [4]:
%timeit -r5 -n10 df_data.iloc[range(0,600)]

195 µs ± 85.9 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


We can see iloc[] perform much faster than loc[] in selecting rows

In [5]:
sub_cols = ["ID", "Case Number", "Date", "Block"]
%timeit -r5 -n10 df_data.loc[:,sub_cols]

1.81 ms ± 947 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [6]:
first3_cols = df_data.columns[:4]
%timeit -r5 -n10 df_data.iloc[:,:4]

1.07 ms ± 353 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


We can see also that using the column indexing using .iloc[] is still faster than loc[]. So it is better to use .iloc[] for efficiency. However, sometimes, it would be more convenient to use .loc[] to select certain columns by name directly

### 2. Iterate through all rows of Dataframe

It is quite common to create a new column based on one or more current columns using Pandas. Think about feature engineering. We can see the different ways for this operation using Pandas. The performances would be compared.

For example, we would like to create a new column which is the sum of square of **X Coordinate** and **Y Coordinate**

In [7]:
sum_square = lambda x, y: (x+y) ** 2
print(sum_square(2,3))

25


In [8]:
test_data = df_data[['X Coordinate', 'Y Coordinate']].copy()

In [9]:
%timeit -r5 -n10 test_data.loc[:,'magic'] = [sum_square(value[0], value[1]) for _, value in test_data.iterrows()]
%timeit -r5 -n10 test_data.loc[:,'magic'] = test_data.apply(lambda row: sum_square(row[0], row[1]), axis=1)
%timeit -r5 -n10 test_data.loc[:,'magic']  = test_data.apply(lambda row: sum_square(row[0], row[1]), raw=True, axis=1)
%timeit -r5 -n10 test_data.loc[:,'magic']  = np.vectorize(sum_square)(test_data.iloc[:,0], test_data.iloc[:,1])
%timeit -r5 -n10 test_data.loc[:,'magic']  = np.power(test_data.iloc[:,0]+test_data.iloc[:,1], 2)
#%timeit -r5 -n10 test_data.loc[:,'magic'] = [sum_square(value[0], value[1]) for _, value in test_data.iterrows()]

402 ms ± 5.84 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
121 ms ± 3.35 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
31.3 ms ± 127 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)
4.85 ms ± 68.1 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)
235 µs ± 26.3 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


The best solution is able to achieve **1700** time speed-up compared to the slowest one  

#### Key Take-away:

1. You should never use iterrows(). If you need to loop through a dataframe, try **itertuples**
2. **raw=True** in the **apply** function is able to  bypass the overhead associated with the Pandas series object. Therefore, it can speed up the apply process.
3. Make the function "vetorized" can yield the highest speed up.

In most of cases, if the operation can be represented as linear algebra operations on matrices and scalar values, it can be vectorized using Numpy methods.

For some operations/functions, it might not be easily vectorized. I prefer parallel processing because it requires the least amount of rewrite of your existing code. You simply have to add a few lines wrapper code to make it work. The example below illustrates how you can do this. For example, we can try **[Dask](https://www.dask.org/)**

In [10]:
!pip install -q dask[dataframe]

zsh:1: no matches found: dask[dataframe]


In [11]:
import dask.dataframe as dd
test_data = df_data[['X Coordinate', 'Y Coordinate']].copy()
ddf = dd.from_pandas(test_data, npartitions=4)
sum_square = lambda row: (row[0]+row[1]) ** 2

In [12]:
%%timeit -r5 -n10
dask_series = ddf.apply(sum_square, axis=1, meta=('magic','float'))
ddf['magic'] = dask_series
df_new = ddf.compute

1.15 ms ± 151 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


The parallelization is able to have a 2.8 times speed up compared to the apply function in the mode of the serial processing.

But for parallel processing, there is a difference between CPU bound (heavy scientific computing and data is in memory) vs I/O bound (making API request over internet). For cpu bound, we should use multiprocess while the task is I/O bound, the multithreading one would be better choice.


### 3. Try to use build-in functions in Pandas

**.groupby()** is one powerful built-in function in Pandas. We can use it to group the entries of a DataFrame according to the values of the specific feature. Then, the following function The .groupby() method is applied to a DataFrame and groups it according to a feature. Then, we can apply some simple or more complicated functions on that grouped object. This is a very important tool for every data scientist working on tabular or structured data as it will help you to manipulate data easily and in a more effective way.

In [13]:
p = 0.2 #percentage missing data required

mask = np.random.choice([np.nan,1], size=len(df_data), p=[p,1-p])
df_datanan = df_data.copy()
df_datanan['Ward'] =  df_datanan['Ward'] * mask

In [14]:
missing_trans = lambda x: x.fillna(x.mean())
df_datanan_grouped = df_datanan.groupby('Primary Type')['Ward']
df_datanan_grouped.transform(missing_trans)

0        22.000000
1         6.000000
2        50.000000
3        48.000000
4        22.216384
           ...    
36077    34.000000
36078    27.000000
36079    24.984009
36080     2.000000
36081    21.925786
Name: Ward, Length: 36082, dtype: float64